In [2]:
def tokenize_function(tokenizer, examples, part, i):
    return (tokenizer(examples[part][i:i+2]["input"], padding="max_length", truncation=True, return_tensors='pt'), 
            tokenizer(examples[part][i:i+2]["output"], padding="max_length", truncation=True, return_tensors='pt'))

In [3]:
from transformers import TrainingArguments, Trainer
import torch
import evaluate
from datasets import load_dataset

from transformers import AutoModelForCausalLM, BartForConditionalGeneration
from transformers import AutoTokenizer

In [4]:
import numpy as np
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [4]:
raw_dataset = load_dataset("tau/scrolls", 'quality')

In [7]:
raw_dataset["train"][0]

{'id': '52995_I3M5VUMM_1',
 'pid': '52995_I3M5VUMM_1_0',
 'input': 'Why is Si retirement so significant to the Space Exploration Team? \n\n (A) There aren’t enough working people in the world. They won’t be able to find a replacement.\n (B) As one of two remaining spacemen, it would likely mean the defunding and shut down of the Space Exploration Team.\n (C) Training new spacemen is costly and time consuming. They won’t have anyone else ready after him.\n (D) His retirement may inspire others to stop working as well, which would be hugely detrimental as most people don\'t feel the drive to work as is.  \n\n\nSPACEMAN ON A SPREE\n\n\n\n\n   BY MACK REYNOLDS\n\n\n\n\n   Illustrated by Nodel\n\n\n\n\n   What\'s more important—Man\'s conquest\n\n\n   of space, or one spaceman\'s life?\n\n\n\n\n\n\n   I\n\n\n\n\n   They gave him a gold watch. It was meant to be symbolical, of course.\n In the old tradition. It was in the way of an antique, being one of the\n timepieces made generations past

In [5]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-large", num_labels=3)

In [6]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
dataset_train = tokenize_function(tokenizer, raw_dataset, 'train', 0)

In [38]:
print(dataset_train)

({'input_ids': tensor([[   0, 7608,   16,  ..., 8099, 6754,    2],
        [   0, 2264,  817,  ...,    6,   10,    2]]), 'attention_mask': tensor([[1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 1, 1, 1]])}, {'input_ids': tensor([[    0, 44466,    92,  ...,     1,     1,     1],
        [    0,   894,   202,  ...,     1,     1,     1]]), 'attention_mask': tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]])})


In [7]:
with torch.no_grad():
    tmp = model(input_ids=dataset_train[0]['input_ids'], attention_mask=dataset_train[0]['attention_mask'], decoder_input_ids=dataset_train[1]['input_ids'], decoder_attention_mask=dataset_train[1]['attention_mask']).logits

In [8]:
tmp.shape

torch.Size([2, 1024, 50265])

In [10]:
from torch.nn import functional
functional.cross_entropy(tmp[:,:-1].reshape(-1, tmp.shape[-1]), dataset_train[1]['input_ids'][:,1:].reshape(-1),reduction="none", ignore_index=tokenizer.pad_token_id).reshape(tmp.shape[0],tmp.shape[1]-1)

tensor([[14.6625, 11.0908,  6.8842,  ...,  0.0000,  0.0000,  0.0000],
        [ 9.2306, 10.2793,  5.5347,  ...,  0.0000,  0.0000,  0.0000]],
       grad_fn=<ReshapeAliasBackward0>)

In [11]:
likelihood_sample = _.sum(1)/(dataset_train[1]['input_ids'][:,1:] != tokenizer.pad_token_id).sum(1)

In [12]:
print(likelihood_sample)

tensor([2.8029, 2.5439], grad_fn=<DivBackward0>)


In [5]:
raw_dataset = load_dataset("tau/scrolls", 'quality')
print(raw_dataset["train"][0])
model = AutoModelForCausalLM.from_pretrained("facebook/bart-large", num_labels=3)
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large")
tokenize_function(tokenizer, raw_dataset[])
dataset_train = tokenize_function(tokenizer, raw_dataset, 'train')
dataset_val = tokenize_function(tokenizer, raw_dataset, 'val')

training_args = TrainingArguments(output_dir="test_training", evaluation_strategy="steps")
trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=dataset_train,
        eval_dataset=dataset_val,
        compute_metrics=compute_metrics,
)

trainer.train()

{'id': '52995_I3M5VUMM_1', 'pid': '52995_I3M5VUMM_1_0', 'input': 'Why is Si retirement so significant to the Space Exploration Team? \n\n (A) There aren’t enough working people in the world. They won’t be able to find a replacement.\n (B) As one of two remaining spacemen, it would likely mean the defunding and shut down of the Space Exploration Team.\n (C) Training new spacemen is costly and time consuming. They won’t have anyone else ready after him.\n (D) His retirement may inspire others to stop working as well, which would be hugely detrimental as most people don\'t feel the drive to work as is.  \n\n\nSPACEMAN ON A SPREE\n\n\n\n\n   BY MACK REYNOLDS\n\n\n\n\n   Illustrated by Nodel\n\n\n\n\n   What\'s more important—Man\'s conquest\n\n\n   of space, or one spaceman\'s life?\n\n\n\n\n\n\n   I\n\n\n\n\n   They gave him a gold watch. It was meant to be symbolical, of course.\n In the old tradition. It was in the way of an antique, being one of the\n timepieces made generations past i

ValueError: text input must of type `str` (single example), `List[str]` (batch or single pretokenized example) or `List[List[str]]` (batch of pretokenized examples).